# Support Vector Machine

## For Binary Classification

### Decision Function

The decision function for a Support Vector Machine (SVM) in binary classification is:


### W<sup>T</sup> X + b = Label


where:

- \( W \) is the weight vector.
- \( X \) is the input vector.
- \( b \) is the bias term.

### Decision Boundary

This equation represents a hyperplane that separates the two classes in feature space.

### Classification:

1. **Positive Class:** If \( W^T X + b > 0 \), the input \( X \) is classified as belonging to one class.
2. **Negative Class:** If \( W^T X + b < 0 \), the input \( X \) is classified as belonging to the other class.

### Objective of SVM

The objective of SVM is to find the hyperplane that maximizes the margin between the two classes. This is done by placing support vectors (key data points) on the margin boundaries to improve classification accuracy and robustness.


### SVM Components

<ol>
<li>HyperPlane</li><br>

<li>Support Vector</li><br>

<li>Margin</li><br>
    
<li>Kernels</li><br>

</ol>


### SVM Kernels

<ol>
<li>Linear</li><br>

<li>Polynomial</li><br>

<li>Radial Basis Function (rbf)</li><br>
    
<li>Sigmoid</li><br>

</ol>


1.**Linear**:

K(x1,x2) = X1<sup>T</sup>X2


2.**Polynomial**:

K(x1,x2) = (X1<sup>T</sup>X2 +r)<sup>d</sup>

3.**Radial Basis Function (rbf)**:

K(x1,x2) = exp(-γ || X1 - X2|| <sup>2</sup>)



4.**Sigmoid Kernel**:

K(x1,x2) = tanh (γ . X<sub>1</sub><sup>T</sup>X<sub>2</sub>+r)

## Loss Function

<p style="color:red">Here we are going to use Hinge Loss</p>

## <mark>L = max(0,1 - y<sub>i</sub>( W<sup>T</sup>.X + b))</mark>

`0 - For Correct`

`1 - For Wrong`

`loss = max(0, 1 - decision_value)
`


## Gradient for SVM Classifier

### if ( y . (W.X - b)>=1) :

dJ/dw = 2 λw

dJ/db = 0 

### else ( y . (W.X - b)<1):


dJ/dw = 2 λw - y.x

dJ/db = y

In [1]:
import numpy as np

In [15]:
class SVM:
    """
    Support Vector Machine (SVM) classifier.

    Attributes:
        learning_rate (float): The learning rate for weight updates.
        num_of_iter (int): The number of iterations for training.
        lambda_parameter (float): The regularization parameter.
        weights (ndarray): The weights of the SVM model.
        bias (float): The bias term of the SVM model.
        rows (int): The number of samples in the training data.
        cols (int): The number of features in the training data.
    """

    def __init__(self, learning_rate=0.01, num_of_iter=1000, lambda_parameter=0.01):
        """
        Initializes the SVM with the given parameters.

        Parameters:
            learning_rate (float): The learning rate for the optimizer.
            num_of_iter (int): The number of iterations for training.
            lambda_parameter (float): The regularization strength.
        """
        self.learning_rate = learning_rate
        self.num_of_iter = num_of_iter
        self.lambda_parameter = lambda_parameter
        self.weights = None
        self.bias = None
        self.rows = None
        self.cols = None

    def fit(self, X, y):
        """
        Fit the SVM model to the training data.

        Parameters:
            X (ndarray): Training data of shape (num_samples, num_features).
            y (ndarray): Target labels of shape (num_samples,), where labels should be 0 or 1.
        """
        self.rows, self.cols = X.shape
        self.weights = np.zeros(self.cols)  # Initialize weights to zero
        self.bias = 0  # Initialize bias to zero

        # Convert labels to -1 and 1
        y_mod = np.where(y == 0, -1, 1)

        # Training loop
        for i in range(self.num_of_iter):
            for index in range(self.rows):
                # Compute the decision value
                decision_value = y_mod[index] * (self.weights @ X[index] - self.bias)

                # Update weights and bias based on the decision value
                if decision_value >= 1:
                    dw = 2 * self.lambda_parameter * self.weights  # Regularization term
                    db = 0
                else:
                    dw = 2 * self.lambda_parameter * self.weights - X[index] * y_mod[index]
                    db = y_mod[index]

                # Update weights and bias
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

    def predict(self, X):
        """
        Predict the class labels for the given input data.

        Parameters:
            X (ndarray): Input data of shape (num_samples, num_features).

        Returns:
            ndarray: Predicted labels (0 or 1) for each input sample.
        """
        # Compute the output
        output = X @ self.weights + self.bias
        
        # Apply the sign function to get predicted labels
        predicted_label = np.sign(output)
        
        # Convert predicted labels from -1 and 1 to 0 and 1
        y_pred = np.where(predicted_label <= -1, 1, 0)
        
        return y_pred


In [3]:
def train_test_split(X, y, test_size=0.2, random_state=None):

  if random_state:
    np.random.seed(random_state)

  indices = list(range(len(X)))
  np.random.shuffle(indices)

  test_size = int(len(X) * test_size)
  test_indices = indices[:test_size]
  train_indices = indices[test_size:]

  X_train, X_test = X[train_indices], X[test_indices]
  y_train, y_test = y[train_indices], y[test_indices]

  return X_train, X_test, y_train, y_test

In [4]:
from sklearn.metrics import accuracy_score
import pandas as pd

In [5]:
df = pd.read_csv("diabetes_prediction_dataset.csv")

In [6]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


### Basic Pre Processing

In [7]:
df.loc[df['gender'] == 'Other', 'gender'] = 'Female'

df['gender'] = pd.get_dummies(df['gender'],drop_first=True).astype(int)

df.drop(columns='smoking_history',inplace=bool(1))

In [8]:
X , x ,Y , y = train_test_split(df.iloc[:,:-1].values,df.iloc[:,-1].values)

In [9]:
model = SVM()

In [10]:
model.fit(X,Y)

In [11]:
y_pred = model.predict(x)

In [13]:
print("The accuraccy of the model is ",accuracy_score(y,y_pred))

The accuraccy of the model is  0.91405
